# AI-Enhanced Qualitative Data Analysis

This notebook processes Excel workbooks containing qualitative data and uses AI models (BERTopic or Top2Vec) to automatically extract themes and topics for qualitative data analysis (QDA).

## Features
- Load multiple Excel sheets/tabs
- Extract qualitative text from specified columns
- Apply BERTopic for hierarchical theme extraction
- Interactive visualizations
- Export results for QDA tools (Taguette, QualCoder, etc.)

## Setup
1. Place your Excel workbook in the `data/raw/` directory
2. Update the `excel_file` path below
3. Customize column names if needed

In [ ]:
# Install required libraries if not already installed
# Uncomment the line below if running in an environment without these packages
# !uv install -r ../requirements.txt

In [ ]:
import pandas as pd
import numpy as np
from bertopic import BERTopic
from umap import UMAP
from hdbscan import HDBSCAN
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML
import warnings
warnings.filterwarnings('ignore')


# Set plotting style
plt.style.use('default')
sns.set_palette("husl")

print("Libraries imported successfully!")

In [ ]:
# Step 1: Load the Excel workbook
# Replace 'your_workbook.xlsx' with the path to your Excel file
excel_file = '../data/raw/callcenterdata.xlsx'

# Load all sheets into a dictionary of DataFrames
try:
    sheets_dict = pd.read_excel(excel_file, sheet_name=None)
    print(f"✓ Successfully loaded workbook: {excel_file}")
    print(f"Loaded sheets: {list(sheets_dict.keys())}")
    
    # Display basic info about each sheet
    for sheet_name, df in sheets_dict.items():
        print(f"  - {sheet_name}: {df.shape[0]} rows, {df.shape[1]} columns")
        
except FileNotFoundError:
    print(f"✗ Error: File '{excel_file}' not found.")
    print("Please place your Excel file in the data/raw/ directory and update the path above.")
    sheets_dict = {}
except Exception as e:
    print(f"✗ Error loading Excel file: {e}")
    sheets_dict = {}

In [ ]:
# Step 2: Extract qualitative text data
# Configure text column names to extract from
text_columns = ['Contact Reason', 'Contact Reason Detail']  # Add your column names here

all_texts = []
text_sources = []  # Track which sheet each text came from

if sheets_dict:
    for sheet_name, df in sheets_dict.items():
        print(f"\nProcessing sheet: {sheet_name}")
        
        # Find text columns in this sheet
        available_text_cols = [col for col in text_columns if col in df.columns]
        
        if available_text_cols:
            print(f"  Found text columns: {available_text_cols}")
            
            # Extract text from each available column
            for col in available_text_cols:
                texts = df[col].dropna().astype(str).tolist()
                all_texts.extend(texts)
                text_sources.extend([f"{sheet_name}:{col}"] * len(texts))
                print(f"    - {col}: {len(texts)} texts extracted")
        else:
            print(f"  Warning: No text columns found in '{sheet_name}'. Available columns: {list(df.columns)}")
    
    print(f"\n✓ Total texts extracted: {len(all_texts)}")
    
    if all_texts:
        print(f"Sample text: {all_texts[0][:100]}...")
        
        # Save processed data
        processed_df = pd.DataFrame({
            'text': all_texts,
            'source': text_sources
        })
        processed_df.to_csv('../data/processed/extracted_texts.csv', index=False)
        print("✓ Processed texts saved to data/processed/extracted_texts.csv")
    else:
        print("✗ No texts were extracted. Please check your column names.")
else:
    print("No workbook loaded. Please check Step 1.")

### Optional: Text Quality Filtering

Before running the AI model, you can filter out texts that are too short or too long. This helps improve topic quality by removing:
- Very short responses (e.g., "N/A", "None", single words)
- Extremely long texts that might be outliers
- Empty or near-empty entries

Adjust `min_text_length` and `max_text_length` based on your data.

In [ ]:
# Step 2.5: Filter texts by quality (optional)
# Remove texts that are too short or likely not meaningful

min_text_length = 10  # Minimum characters per text
max_text_length = 10000  # Maximum characters (remove outliers)

if all_texts:
    original_count = len(all_texts)
    
    # Filter texts and their sources together
    filtered_data = [
        (text, source) for text, source in zip(all_texts, text_sources)
        if min_text_length <= len(str(text)) <= max_text_length
    ]
    
    if filtered_data:
        all_texts, text_sources = zip(*filtered_data)
        all_texts = list(all_texts)
        text_sources = list(text_sources)
        
        removed_count = original_count - len(all_texts)
        print(f"✓ Text filtering complete:")
        print(f"  - Kept: {len(all_texts)} texts")
        print(f"  - Removed: {removed_count} texts (too short or too long)")
        print(f"  - Length range: {min_text_length}-{max_text_length} characters")
        
        # Show distribution
        lengths = [len(str(t)) for t in all_texts]
        print(f"  - Average length: {np.mean(lengths):.0f} characters")
        print(f"  - Median length: {np.median(lengths):.0f} characters")
    else:
        print("✗ Warning: All texts were filtered out. Try adjusting min/max length.")
else:
    print("No texts to filter.")

In [ ]:
# Step 3: Apply BERTopic for theme extraction
model_choice = globals().get('model_choice', 'bertopic')

if all_texts:
    print(f"Running {model_choice.upper()} analysis on {len(all_texts)} texts...")
    
    # Handle large datasets by sampling
    if len(all_texts) > 50000:
        print(f"Dataset is large ({len(all_texts)} texts). Sampling to 50,000 for analysis to avoid memory issues.")
        sample_indices = np.random.choice(len(all_texts), 50000, replace=False)
        analysis_texts = [all_texts[i] for i in sample_indices]
        analysis_sources = [text_sources[i] for i in sample_indices]
    else:
        analysis_texts = all_texts
        analysis_sources = text_sources

    # Import additional libraries for custom models
    from umap import UMAP
    from hdbscan import HDBSCAN
    
    # Initialize BERTopic model with optimized settings for large datasets
    topic_model = BERTopic(
        language="english",
        calculate_probabilities=False,  # Set to False to reduce memory usage
        verbose=True,
        min_topic_size=10,  # Increased from 5 to reduce number of topics and memory
        nr_topics="auto",   # Auto-determine number of topics
        low_memory=True,  # Enable low memory mode for large datasets
        umap_model=UMAP(n_jobs=1),  # Disable multiprocessing in UMAP
        hdbscan_model=HDBSCAN(core_dist_n_jobs=1)  # Disable multiprocessing in HDBSCAN
    )
    
    # Fit the model
    topics, probabilities = topic_model.fit_transform(analysis_texts)
    
    # Get topic information
    theme_info = topic_model.get_topic_info()
    
    print(f"\n✓ BERTopic identified {len(theme_info)-1} topics (excluding outliers)")
    
    # Display extracted themes
    display(theme_info.head(10))
    
else:
    print("No texts available for analysis. Please check Step 2.")
    theme_info = None

In [ ]:
# Step 4: Visualize topics
# Safely get variables that may not exist in the current kernel/session
theme_info = globals().get('theme_info', None)
model_choice = globals().get('model_choice', 'bertopic')
topic_model = globals().get('topic_model', None)

from IPython.display import display, HTML
import plotly.io as pio

# Prefer a non-blocking renderer for notebooks; HTML embedding fallback is safest
try:
    pio.renderers.default = 'notebook_connected'
except Exception:
    pass

def _display_fig(fig, title=None):
    """Embed a Plotly figure as HTML to avoid renderer blocking issues."""
    try:
        html = fig.to_html(full_html=False, include_plotlyjs='cdn')
        if title:
            display(HTML(f"<h4>{title}</h4>"))
        display(HTML(html))
    except Exception as e:
        print(f"Failed to render figure as HTML: {e}")

if theme_info is not None and topic_model is not None:
    print("Generating visualizations...")
    
    # Topic visualization
    try:
        fig_topics = topic_model.visualize_topics()
        _display_fig(fig_topics, title='Topic Overview')
    except Exception as e:
        print(f"Topic visualization failed: {e}")
    
    # Topic hierarchy (if enough topics and not too many)
    try:
        if len(theme_info) > 3 and len(theme_info) < 200:
            fig_hierarchy = topic_model.visualize_hierarchy()
            _display_fig(fig_hierarchy, title='Topic Hierarchy')
        else:
            if len(theme_info) >= 200:
                print('Skipping hierarchy visualization: too many topics to render.')
    except Exception as e:
        print(f"Hierarchy visualization failed: {e}")
    
    # Topic distribution
    try:
        fig_barchart = topic_model.visualize_barchart(top_n_topics=10)
        _display_fig(fig_barchart, title='Top Topics Barchart')
    except Exception as e:
        print(f"Barchart visualization failed: {e}")
    
else:
    if theme_info is None:
        print("No themes available for visualization. Run the analysis cells first.")
    elif model_choice == 'bertopic' and topic_model is None:
        print("BERTopic model object not found in the session. Re-run the BERTopic analysis cell.")
    else:
        print("No visualizations available for the current configuration.")

In [ ]:
# Step 5: Export themes for validation/import into QDA tool
if theme_info is not None:
    # Export to CSV
    output_file = f"../data/results/extracted_themes_{model_choice}.csv"
    theme_info.to_csv(output_file, index=False)
    print(f"✓ Themes exported to '{output_file}'")
    
    # Export detailed topic information
    # Get topic representations
    topic_representations = {}
    for topic_id in theme_info['Topic'].unique():
        if topic_id != -1:  # Skip outlier topic
            words = topic_model.get_topic(topic_id)
            topic_representations[topic_id] = words
    
    # Create detailed export
    detailed_themes = []
    for topic_id, words in topic_representations.items():
        detailed_themes.append({
            'topic_id': topic_id,
            'topic_name': f"Topic_{topic_id}",
            'top_words': ', '.join([word for word, _ in words[:10]]),
            'word_scores': ', '.join([f"{word}:{score:.3f}" for word, score in words[:10]]),
            'document_count': theme_info[theme_info['Topic'] == topic_id]['Count'].iloc[0]
        })
    
    detailed_df = pd.DataFrame(detailed_themes)
    detailed_output = f"../data/results/detailed_themes_{model_choice}.csv"
    detailed_df.to_csv(detailed_output, index=False)
    print(f"✓ Detailed themes exported to '{detailed_output}'")
    
    # Export text-topic assignments for QDA validation
    text_topics_df = pd.DataFrame({
        'text': analysis_texts,
        'source': analysis_sources,
        'topic': topics,
        'topic_probability': [None] * len(topics)  # Probabilities disabled for memory efficiency
    })

    
    assignments_file = f"../data/results/text_topic_assignments_{model_choice}.csv"
    text_topics_df.to_csv(assignments_file, index=False)
    print(f"✓ Text-topic assignments exported to '{assignments_file}'")
    
    print("\n" + "="*50)
    print("EXPORT COMPLETE")
    print("="*50)
    print("Files ready for import into QDA tools like:")
    print("- Taguette (import CSV files)")
    print("- QualCoder (import CSV files)")
    print("- NVivo (import CSV files)")
    print("- ATLAS.ti (import CSV files)")
    
else:
    print("No themes to export.")

## Next Steps

1. **Validate AI-generated themes** in your QDA tool
2. **Refine topics** by merging, splitting, or renaming as needed
3. **Code additional texts** using the validated themes
4. **Export coded data** for further quantitative analysis

## Tips for Better Results

- **Preprocessing**: Clean your text data before analysis (remove noise, standardize formats)
- **Model Tuning**: Adjust `min_topic_size` in BERTopic for more/fewer topics
- **Language**: Set the correct language if your texts aren't in English
- **Scale**: For large datasets (>10k texts), consider running on Databricks Community Edition

## Troubleshooting

- **Memory issues**: Reduce `min_topic_size` or use Top2Vec instead
- **Poor topics**: Preprocess text better or try different model settings
- **No texts found**: Check column names in your Excel file